In [ ]:
import os

# Define the dataset name (change this to your dataset)
dataset_name = "danaelisanicolas/high-density-crowd-counting"

# Define the output folder
output_folder = "HighDensityCrowd_Dataset"

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Download dataset using Kaggle API
print("Downloading dataset from Kaggle...")
os.system(f"kaggle datasets download -d {dataset_name} -p {output_folder} --unzip")
print("Download & extraction complete!")


In [ ]:
import requests
import zipfile
import os

# Ensure the Dropbox link uses dl=1 for direct download
dropbox_url = "https://www.dropbox.com/scl/fi/dkj5kulc9zj0rzesslck8/ShanghaiTech_Crowd_Counting_Dataset.zip?dl=1"
output_file = "ShanghaiTech_Crowd_Counting_Dataset.zip"

# Re-download the file to avoid corruption
print("Downloading ShanghaiTech dataset from Dropbox...")
response = requests.get(dropbox_url, stream=True)

# Save the downloaded content
with open(output_file, "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)
print("Download complete!")

# 🔹 Check if the file is really a ZIP file before extracting
if zipfile.is_zipfile(output_file):
    print("File is a valid ZIP, extracting...")
    extract_folder = "ShanghaiTech_Dataset"
    
    with zipfile.ZipFile(output_file, "r") as zip_ref:
        zip_ref.extractall(extract_folder)
    
    print("Extraction complete!")
else:
    print("⚠ ERROR: The downloaded file is not a ZIP! It may be an HTML error page.")


In [ ]:
import subprocess

# GitHub repository URL
github_repo_url = "https://github.com/mchengny/RWF2000-Video-Database-for-Violence-Detection.git"

# GitHub output folder
github_output_folder = "RWF2000_Violence_Dataset"

# Clone the repository
print("Cloning RWF-2000 dataset from GitHub...")
subprocess.run(["git", "clone", github_repo_url, github_output_folder])
print("Cloning complete!")


In [ ]:
import fiftyone as fo

# Load the existing dataset
dataset = fo.load_dataset("stampede_detection")

# Compute metadata for all images
dataset.compute_metadata()

# Save the dataset
dataset.save()

print("✅ Metadata added successfully!")


In [ ]:
import fiftyone as fo
import random

# Load the existing dataset
dataset = fo.load_dataset("stampede_detection")

# Define possible labels based on the constraints
labels = ["Crowd Density", "Motion", "People Falling", "Behavior Shift", "Panic/Aggression"]

# Function to assign random labels for now (Replace with actual label logic)
def assign_labels(sample):
    assigned_labels = []

    # Simulate conditions (Replace with actual detection logic)
    if random.random() > 0.5:
        assigned_labels.append("Crowd Density")
    if random.random() > 0.5:
        assigned_labels.append("Motion")
    if random.random() > 0.3:
        assigned_labels.append("People Falling")
    if random.random() > 0.4:
        assigned_labels.append("Behavior Shift")
    if random.random() > 0.2:
        assigned_labels.append("Panic/Aggression")

    return assigned_labels if assigned_labels else ["Normal"]

# Add labels to dataset
for sample in dataset:
    sample["ground_truth"] = fo.Classifications(
        classifications=[fo.Classification(label=label) for label in assign_labels(sample)]
    )
    sample.save()

# Save dataset after adding labels
dataset.save()
print("✅ Labels added successfully!")

# Relaunch FiftyOne to check labels
session = fo.launch_app(dataset)


In [ ]:
import os
import glob
from PIL import Image
import fiftyone as fo

# Define paths
gif_folder = "RWF2000_Violence_Dataset/Images"
output_folder = "gif_frames"
os.makedirs(output_folder, exist_ok=True)


frame_count = 0
samples = []

# Get all GIF files
gif_files = glob.glob(os.path.join(gif_folder, "*.gif"))

if not gif_files:
    print("⚠ ERROR: No GIF files found!")
else:
    print(f"✅ Found {len(gif_files)} GIFs.")

for gif_file in gif_files:
    gif_name = os.path.basename(gif_file).split(".")[0]  # Extract file name without extension

    # Open GIF and extract frames
    with Image.open(gif_file) as img:
        for frame in range(img.n_frames):
            img.seek(frame)
            frame_path = os.path.join(output_folder, f"{gif_name}_frame_{frame}.jpg")
            img.convert("RGB").save(frame_path)

            # Add frame to FiftyOne dataset
            sample = fo.Sample(filepath=frame_path)
            samples.append(sample)

            frame_count += 1

# Add extracted frames to FiftyOne
if samples:
    dataset.add_samples(samples)
    print(f"✅ Extracted {frame_count} frames from GIFs and added to FiftyOne dataset.")
else:
    print("⚠ ERROR: No frames were extracted!")

# Launch FiftyOne for visualization
if frame_count > 0:
    session = fo.launch_app(dataset)


In [ ]:
import fiftyone as fo

# Load your dataset
dataset_name = "stampede_detection"
dataset = fo.load_dataset(dataset_name)

# Print dataset summary
print(dataset)
print("Total Samples:", len(dataset))

# Check first few samples
print(dataset.first())


In [ ]:
# Launch annotation tool
session = fo.launch_app(dataset)

# Save annotations after manual labeling
dataset.save()


In [ ]:
export_dir = "yolo_dataset"
dataset.export(
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv5Dataset,
    label_field="ground_truth"
)
print(f"Dataset exported to {export_dir}")


In [ ]:
from ultralytics import YOLO

# Train YOLO on labeled dataset
model = YOLO("yolov8n.pt")  # Using YOLOv8 Nano for efficiency
model.train(
    data="yolo_dataset/data.yaml",
    epochs=50,
    imgsz=640
)


In [ ]:
import cv2
from ultralytics import YOLO

# Load trained YOLO model
model = YOLO("runs/train/exp/weights/best.pt")

# Start webcam or use video file
cap = cv2.VideoCapture(0)  # Use 0 for webcam, or replace with video file

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform YOLO inference
    results = model(frame)

    # Draw detections
    for result in results:
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box[:4])
            label = result.names[int(box[4])]
            confidence = result.boxes.conf[0].item()

            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label}: {confidence:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show video
    cv2.imshow("Stampede Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
